We are working with a semi structured JSON-style dataset in PySpark.where, each records contains user_id,user_name,purchases(an array of struct,each with:
item_name,quantity,price)

our task is to flattern the purchases array so that each item becomes a seperate row also to calculate the total_cost per item(quantity X price).

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode,col,expr

In [3]:
spark=SparkSession.builder.appName("FlatternStructArray").getOrCreate()

In [14]:
data=([("ABC1", "Alice", [
 {"item_name": "Book", "quantity": 2, "price": 300},
 {"item_name": "Pen", "quantity": 5, "price": 20}
 ]),
 ("ABC2", "Bob", [
 {"item_name": "Notebook", "quantity": 1, "price": 150}
 ])])

In [15]:
data

[('ABC1',
  'Alice',
  [{'item_name': 'Book', 'quantity': 2, 'price': 300},
   {'item_name': 'Pen', 'quantity': 5, 'price': 20}]),
 ('ABC2', 'Bob', [{'item_name': 'Notebook', 'quantity': 1, 'price': 150}])]

In [22]:
schema="user_id STRING,user_name STRING,purchases ARRAY<STRUCT<item_name:STRING,quantity:INT,price:INT>>"
schema

'user_id STRING,user_name STRING,purchases ARRAY<STRUCT<item_name:STRING,quantity:INT,price:INT>>'

In [23]:
df=spark.createDataFrame(data,schema=schema)
df

Explore the array of structs

In [24]:
flat_df=df.withColumn("item",explode("purchases")).select("user_id","user_name",col("item.item_name"),col("item.quantity").alias("qty"),col("item.price"),(col("item.quantity")*col("item.price")).alias("total_cost"))

In [25]:
flat_df

DataFrame[user_id: string, user_name: string, item_name: string, qty: int, price: int, total_cost: int]

In [26]:
flat_df.show()

+-------+---------+---------+---+-----+----------+
|user_id|user_name|item_name|qty|price|total_cost|
+-------+---------+---------+---+-----+----------+
|   ABC1|    Alice|     Book|  2|  300|       600|
|   ABC1|    Alice|      Pen|  5|   20|       100|
|   ABC2|      Bob| Notebook|  1|  150|       150|
+-------+---------+---------+---+-----+----------+

